In [19]:
import cv2
from PIL import Image, ImageDraw
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1

In [4]:
def capture_and_save_owner_embedding():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)
    resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        
        # Convert to RGB and detect faces
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(frame_rgb)
        boxes, _ = mtcnn.detect(pil_img)
        
        if boxes is not None:
            # Extract the face with the largest area (most likely the main subject)
            areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in boxes]
            largest_face_index = areas.index(max(areas))
            largest_face = mtcnn.extract(pil_img, [boxes[largest_face_index]], None)
            
            # Save the embedding of the largest detected face
            owner_embedding = resnet(largest_face).detach()
            torch.save(owner_embedding, 'owner_embedding.pt')
            
            print("Owner's face embedding captured and saved.")
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return owner_embedding 

In [6]:
capture_and_save_owner_embedding()

100%|██████████| 107M/107M [05:00<00:00, 373kB/s]  


Owner's face embedding captured and saved.


tensor([[ 0.0021,  0.0769,  0.0299,  0.0142, -0.0086,  0.0093,  0.0047,  0.0161,
          0.0467, -0.0544,  0.0642, -0.0252,  0.0319,  0.0242, -0.0216, -0.0483,
         -0.0527, -0.0005, -0.0003,  0.0345,  0.0481, -0.0669,  0.0050, -0.0214,
          0.0328,  0.0082,  0.0270, -0.0053, -0.0034, -0.0353,  0.0330,  0.0362,
         -0.0751,  0.0268,  0.0725,  0.0279, -0.0073,  0.0073, -0.0289,  0.0258,
         -0.0248,  0.0031, -0.0645, -0.0327, -0.0332, -0.0751, -0.0350,  0.0339,
          0.0595, -0.0430, -0.0728, -0.0725,  0.0206,  0.0622,  0.0904,  0.0462,
         -0.0201,  0.0223,  0.0122, -0.0033,  0.0642, -0.0570,  0.0424,  0.0551,
          0.0909, -0.0426, -0.0244, -0.0571,  0.0066, -0.0536, -0.1010, -0.0079,
         -0.0637, -0.0327,  0.0964, -0.0487, -0.0153, -0.0088,  0.0632, -0.0170,
         -0.0398, -0.0196, -0.0037, -0.0488, -0.0177,  0.0504,  0.0180, -0.0062,
          0.0564,  0.0116,  0.0069,  0.0322, -0.0487, -0.0394, -0.0352,  0.0449,
          0.0593, -0.0137,  

In [23]:
def recognize_and_highlight():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)
    resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
    owner_embedding = torch.load('owner_embedding.pt')

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)

            # Detect faces
            boxes, _ = mtcnn.detect(pil_img)
            draw = ImageDraw.Draw(pil_img)
            if boxes is not None:
                faces = mtcnn.extract(pil_img, boxes, None)
                embeddings = resnet(faces).detach()

                for i, box in enumerate(boxes):
                    # Calculate distance to the owner's embedding
                    distance = (embeddings[i] - owner_embedding).norm().item()
                    if distance < 0.6:  # threshold for recognition, tune based on your dataset
                        outline_color = (0, 255, 0)  # Green for owner
                    else:
                        outline_color = (255, 0, 0)  # Red for others

                    draw.rectangle(box.tolist(), outline=outline_color, width=6)

            # Display the image
            cv_frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
            cv2.imshow('Webcam', cv_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [24]:
recognize_and_highlight()

In [22]:
def is_owner_present():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)
    resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
    owner_embedding = torch.load('owner_embedding.pt')

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    owner_detected = False  # Flag to indicate if the owner is detected

    try:
        # Capture a single frame
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)

            # Detect faces
            boxes, _ = mtcnn.detect(pil_img)
            if boxes is not None:
                faces = mtcnn.extract(pil_img, boxes, None)
                embeddings = resnet(faces).detach()

                for embedding in embeddings:
                    # Calculate distance to the owner's embedding
                    distance = (embedding - owner_embedding).norm().item()
                    if distance < 0.6:  # threshold for recognition, tune based on your dataset
                        owner_detected = True
                        break

        # Optionally, display the frame with or without marking
        # Comment out if not needed
        cv_frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        cv2.imshow('Webcam', cv_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            pass

    finally:
        cap.release()
        cv2.destroyAllWindows()

    return owner_detected

# Example of using the function
owner_present = is_owner_present()
print("Owner present:", owner_present)

Owner present: False
